In [ ]:
!pip install datasets nltk transformers[sentencepiece] sacrebleu -q
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
import tensorflow as tf
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamWeightDecay

import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_excel('/content/translated_eng_to_malayalam.xlsx')  # Assuming columns: 'hindi', 'english'

# Convert the dataframe into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Train-test split
raw_datasets = dataset.train_test_split(test_size=0.2)

In [ ]:
# Model checkpoint
model_checkpoint = "Helsinki-NLP/opus-mt-ml-en"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define max sequence lengths
max_input_length = 128
max_target_length = 128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/818k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.72M [00:00<?, ?B/s]

In [ ]:
# Define source and target languages
source_lang = "Malayalam"  # Column name for Hindi
target_lang = "English"  # Column name for English

In [ ]:
# Preprocessing function
def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/3724 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/932 [00:00<?, ? examples/s]

In [ ]:
# Load the pre-trained model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/309M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-ml-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# Set hyperparameters
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 5  # You can adjust this

In [ ]:
# Data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

# Prepare datasets for TensorFlow
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

# Define optimizer
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

In [ ]:
# Compile the model
model.compile(optimizer=optimizer)

In [ ]:
# Fine-tune the model
model.fit(train_dataset, validation_data=validation_dataset, epochs=num_train_epochs)
# Save the fine-tuned model
model.save_pretrained("tf_ml_to_en_modell/")
tokenizer.save_pretrained("tf_ml_to_en_modell/")

!mkdir -p saved_translator_modell

# Save model and tokenizer
model.save_pretrained("saved_translator_modell")
tokenizer.save_pretrained("saved_translator_modell")

# Zip the model files for easy download
!zip -r translator_modell.zip saved_translator_modell/

# Automatically download the zip file
from google.colab import files
files.download("translator_modell.zip")

Epoch 1/5
232/232 [==============================] - 4149s 18s/step - loss: 3.5336 - val_loss: 2.8634
Epoch 2/5
232/232 [==============================] - 4076s 18s/step - loss: 2.8191 - val_loss: 2.5452
Epoch 3/5
232/232 [==============================] - 4077s 18s/step - loss: 2.4540 - val_loss: 2.3516
Epoch 4/5
232/232 [==============================] - 4115s 18s/step - loss: 2.1960 - val_loss: 2.2193
Epoch 5/5
232/232 [==============================] - 4084s 18s/step - loss: 1.9841 - val_loss: 2.1202


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[63223]]}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:388: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in th

  adding: saved_translator_modell/ (stored 0%)
  adding: saved_translator_modell/tf_model.h5 (deflated 7%)
  adding: saved_translator_modell/vocab.json (deflated 80%)
  adding: saved_translator_modell/target.spm (deflated 52%)
  adding: saved_translator_modell/generation_config.json (deflated 43%)
  adding: saved_translator_modell/special_tokens_map.json (deflated 35%)
  adding: saved_translator_modell/config.json (deflated 61%)
  adding: saved_translator_modell/tokenizer_config.json (deflated 68%)
  adding: saved_translator_modell/source.spm (deflated 65%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>